In [2]:
import json
import random
import datetime
from faker import Faker
import psycopg2
import uuid
import requests
import os
from io import BytesIO

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle

In [3]:
fake = Faker()

In [4]:
def get_db_connection():
    return psycopg2.connect(
        host='localhost',
        port='5433',
        user='admin',
        password='password',
        database='university'
    )

def load_student_ids():
    student_ids = []
    try:
        conn = get_db_connection()
        cur = conn.cursor()
        cur.execute("SELECT student_id FROM students")
        student_ids = [row[0] for row in cur.fetchall()]
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Error loading student IDs: {e}")
    
    return student_ids

def load_student_names(student_ids):
    student_names = []
    names_map = dict()
    try:
        conn = get_db_connection()
        cur = conn.cursor()
        cur.execute("SELECT name FROM students")
        student_names = [row[0] for row in cur.fetchall()]
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Error loading student names: {e}")
    
    for i, student_id in enumerate(student_ids):
        names_map[student_id] = student_names[i]
    return names_map

def load_addresses(student_ids):
    addresses = []
    addresses_map = dict()
    try:
        conn = get_db_connection()
        cur = conn.cursor()
        cur.execute("SELECT address FROM students")
        addresses = [row[0] for row in cur.fetchall()]
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Error loading student addresses: {e}")

    for i, student_id in enumerate(student_ids):
        addresses_map[student_id] = addresses[i]

    return addresses_map

def load_emails(student_ids):
    emails = []
    emails_map = dict()
    try:
        conn = get_db_connection()
        cur = conn.cursor()
        cur.execute("SELECT email FROM students")
        emails = [row[0] for row in cur.fetchall()]
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Error loading student addresses: {e}")

    for i, student_id in enumerate(student_ids):
        emails_map[student_id] = emails[i]

    return emails_map

In [5]:
student_ids = load_student_ids()
names = load_student_names(student_ids)
addresses = load_addresses(student_ids)
emails = load_emails(student_ids)
len(student_ids), len(names), len(addresses), len(emails)

(9000, 9000, 9000, 9000)

In [6]:
def generate_tax_return(student_info):
    buffer = BytesIO()
    doc = SimpleDocTemplate(buffer, pagesize=letter)
    styles = getSampleStyleSheet()
    
    # Content elements
    elements = []
    
    # Title
    tax_year = student_info.get('tax_year', datetime.datetime.now().year - 1)
    elements.append(Paragraph(f"FORM 1040 - INDIVIDUAL INCOME TAX RETURN {tax_year}", styles['Title']))
    
    # Basic info
    elements.append(Paragraph(f"Name: {names[student_info['student_id']]}", styles['Normal']))
    elements.append(Paragraph(f"Address: {addresses[student_info['student_id']]}", styles['Normal']))
    elements.append(Spacer(1, 12))
    
    # Income section
    elements.append(Paragraph("INCOME", styles['Heading2']))
    
    # Random income data
    wages = random.randint(5000, 35000)
    interest = random.randint(0, 500)
    other_income = random.randint(0, 3000)
    total_income = wages + interest + other_income
    
    income_data = [
        ["Category", "Amount"],
        ["Wages, salaries, tips", f"${wages}"],
        ["Taxable interest", f"${interest}"],
        ["Other income", f"${other_income}"],
        ["TOTAL INCOME", f"${total_income}"]
    ]
    
    income_table = Table(income_data, colWidths=[300, 100])
    income_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (1, 0), colors.lightgrey),
        ('TEXTCOLOR', (0, 0), (1, 0), colors.black),
        ('ALIGN', (0, 0), (1, 0), 'CENTER'),
        ('FONTNAME', (0, 0), (1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (1, 0), 12),
        ('BACKGROUND', (0, -1), (1, -1), colors.lightgrey),
        ('FONTNAME', (0, -1), (1, -1), 'Helvetica-Bold'),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    
    elements.append(income_table)
    elements.append(Spacer(1, 24))
    
    # Tax calculation
    tax = total_income * 0.12  # Simplified tax calculation
    elements.append(Paragraph("TAX CALCULATION", styles['Heading2']))
    
    tax_data = [
        ["Category", "Amount"],
        ["Total income", f"${total_income}"],
        ["Tax (simplified rate)", f"${int(tax)}"],
        ["Tax payments withheld", f"${int(tax * 1.1)}"],  # Slightly overpaid
        ["Refund amount", f"${int(tax * 0.1)}"]  # Refund is the difference
    ]
    
    tax_table = Table(tax_data, colWidths=[300, 100])
    tax_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (1, 0), colors.lightgrey),
        ('TEXTCOLOR', (0, 0), (1, 0), colors.black),
        ('ALIGN', (0, 0), (1, 0), 'CENTER'),
        ('FONTNAME', (0, 0), (1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (1, 0), 12),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    
    elements.append(tax_table)
    elements.append(Spacer(1, 36))
    
    # Signature
    elements.append(Paragraph(f"Signature: ___________________ Date: {fake.date_this_year().strftime('%m/%d/%Y')}", styles['Normal']))
    
    # Build PDF
    doc.build(elements)
    
    return buffer.getvalue()

In [7]:
def generate_fafsa(student_info):
    buffer = BytesIO()
    doc = SimpleDocTemplate(buffer, pagesize=letter)
    styles = getSampleStyleSheet()
    
    # Content elements
    elements = []
    
    # Title
    aid_year = f"{datetime.datetime.now().year}/{datetime.datetime.now().year + 1}"
    elements.append(Paragraph(f"Free Application for Federal Student Aid (FAFSA) - {aid_year}", styles['Title']))
    
    # Student information
    elements.append(Paragraph("STUDENT INFORMATION", styles['Heading2']))
    elements.append(Paragraph(f"Name: {names[student_info['student_id']]}", styles['Normal']))
    elements.append(Paragraph(f"Date of Birth: {fake.date_of_birth(minimum_age=18, maximum_age=30).strftime('%m/%d/%Y')}", styles['Normal']))
    elements.append(Paragraph(f"Email: {emails[student_info['student_id']]}", styles['Normal']))
    elements.append(Paragraph(f"Phone: {fake.phone_number()}", styles['Normal']))
    elements.append(Paragraph(f"Address: {addresses[student_info['student_id']]}", styles['Normal']))
    elements.append(Spacer(1, 12))
    
    # Financial information
    elements.append(Paragraph("FINANCIAL INFORMATION", styles['Heading2']))
    
    # Generate EFC (Expected Family Contribution)
    efc = random.randint(0, 20000)
    dependency = random.choice(["Dependent", "Independent"])
    
    financial_data = [
        ["Category", "Amount/Status"],
        ["Student Income", f"${student_info.get('income', random.randint(5000, 35000))}"],
        ["Parent Income", f"${random.randint(30000, 120000)}"],
        ["Household Size", str(random.randint(2, 6))],
        ["Dependency Status", dependency],
        ["Expected Family Contribution (EFC)", f"${efc}"]
    ]
    
    financial_table = Table(financial_data, colWidths=[300, 100])
    financial_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (1, 0), colors.lightgrey),
        ('TEXTCOLOR', (0, 0), (1, 0), colors.black),
        ('ALIGN', (0, 0), (1, 0), 'CENTER'),
        ('FONTNAME', (0, 0), (1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (1, 0), 12),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    
    elements.append(financial_table)
    elements.append(Spacer(1, 24))
    
    # Schools section
    elements.append(Paragraph("SCHOOLS", styles['Heading2']))
    elements.append(Paragraph("The following schools will receive your FAFSA information:", styles['Normal']))
    
    school_list = []
    for _ in range(random.randint(1, 4)):
        school_list.append(f"• {fake.company()} University - School Code: {fake.numerify('###')}###")
    
    for school in school_list:
        elements.append(Paragraph(school, styles['Normal']))
    
    elements.append(Spacer(1, 36))
    
    # Signature
    elements.append(Paragraph(f"Student Signature: ___________________ Date: {fake.date_this_year().strftime('%m/%d/%Y')}", styles['Normal']))
    if dependency == "Dependent":
        elements.append(Paragraph(f"Parent Signature: ___________________ Date: {fake.date_this_year().strftime('%m/%d/%Y')}", styles['Normal']))
    
    # Build PDF
    doc.build(elements)
    
    return buffer.getvalue()

In [8]:
def generate_scholarship_application(student_info):
    buffer = BytesIO()
    doc = SimpleDocTemplate(buffer, pagesize=letter)
    styles = getSampleStyleSheet()
    
    # Content elements
    elements = []
    
    # Title - pick a random scholarship name
    scholarship_names = [
        "Merit Achievement Scholarship",
        "Community Leadership Award",
        "Academic Excellence Scholarship",
        "Future Leaders Grant",
        "STEM Success Scholarship",
        "Diversity in Education Award",
        "First Generation College Fund"
    ]
    scholarship_name = random.choice(scholarship_names)
    
    elements.append(Paragraph(f"{scholarship_name} Application", styles['Title']))
    
    # Student information
    elements.append(Paragraph("APPLICANT INFORMATION", styles['Heading2']))
    elements.append(Paragraph(f"Name: {names[student_info['student_id']]}", styles['Normal']))
    elements.append(Paragraph(f"Student ID: {student_info['student_id']}", styles['Normal']))
    elements.append(Paragraph(f"Email: {emails[student_info['student_id']]}", styles['Normal']))
    elements.append(Paragraph(f"Program: {student_info.get('program', 'Bachelor of Science')}", styles['Normal']))
    elements.append(Paragraph(f"GPA: {round(random.uniform(2.5, 4.0), 2)}", styles['Normal']))
    elements.append(Spacer(1, 12))
    
    # Essay/Personal Statement
    elements.append(Paragraph("PERSONAL STATEMENT", styles['Heading2']))
    
    essay_prompt = random.choice([
        "Describe how this scholarship will help you achieve your academic goals.",
        "Explain your leadership experience and how it has shaped your career aspirations.",
        "Discuss a challenge you've overcome and what you learned from the experience."
    ])
    
    elements.append(Paragraph(f"Prompt: {essay_prompt}", styles['Normal']))
    elements.append(Spacer(1, 12))
    
    # Generate fake essay paragraphs
    essay_paragraphs = []
    for _ in range(5):
        essay_paragraphs.append(fake.paragraph(nb_sentences=6))
    
    for paragraph in essay_paragraphs:
        elements.append(Paragraph(paragraph, styles['Normal']))
        elements.append(Spacer(1, 6))
    
    elements.append(Spacer(1, 24))
    
    # References
    elements.append(Paragraph("REFERENCES", styles['Heading2']))
    
    for _ in range(2):
        ref_name = fake.name()
        ref_title = random.choice(["Professor", "Advisor", "Mentor", "Employer", "Coach"])
        ref_phone = fake.phone_number()
        ref_email = fake.email()
        
        elements.append(Paragraph(f"Name: {ref_name}", styles['Normal']))
        elements.append(Paragraph(f"Title: {ref_title}", styles['Normal']))
        elements.append(Paragraph(f"Phone: {ref_phone}", styles['Normal']))
        elements.append(Paragraph(f"Email: {ref_email}", styles['Normal']))
        elements.append(Spacer(1, 12))
    
    # Signature
    elements.append(Spacer(1, 24))
    elements.append(Paragraph(f"Signature: ___________________ Date: {fake.date_this_year().strftime('%m/%d/%Y')}", styles['Normal']))
    
    # Build PDF
    doc.build(elements)
    
    return buffer.getvalue()


In [9]:
def generate_verification_document(student_info):
    buffer = BytesIO()
    doc = SimpleDocTemplate(buffer, pagesize=letter)
    styles = getSampleStyleSheet()
    
    # Content elements
    elements = []
    
    # Title
    elements.append(Paragraph("FINANCIAL AID VERIFICATION WORKSHEET", styles['Title']))
    
    # Student information
    elements.append(Paragraph("STUDENT INFORMATION", styles['Heading2']))
    elements.append(Paragraph(f"Name: {names[student_info['student_id']]}", styles['Normal']))
    elements.append(Paragraph(f"Student ID: {student_info['student_id']}", styles['Normal']))
    elements.append(Paragraph(f"Academic Year: {datetime.datetime.now().year}/{datetime.datetime.now().year + 1}", styles['Normal']))
    elements.append(Spacer(1, 12))
    
    # Household information
    elements.append(Paragraph("HOUSEHOLD INFORMATION", styles['Heading2']))
    
    household_size = random.randint(2, 6)
    in_college = random.randint(1, min(household_size, 3))
    
    household_data = [
        ["Family Member", "Age", "Relationship", "College (if attending)"],
        [names[student_info['student_id']], str(random.randint(18, 25)), "Self", "This Institution"]
    ]
    
    # Add additional family members
    for i in range(1, household_size):
        is_in_college = (i < in_college)
        relationship = random.choice(["Parent", "Sibling", "Spouse"]) if i < 3 else "Sibling"
        age = random.randint(18, 22) if relationship == "Sibling" else random.randint(40, 60)
        college = fake.company() + " University" if is_in_college and relationship != "Parent" else "N/A"
        
        family_member = fake.name()
        household_data.append([family_member, str(age), relationship, college])
    
    household_table = Table(household_data, colWidths=[150, 50, 100, 150])
    household_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (3, 0), colors.lightgrey),
        ('TEXTCOLOR', (0, 0), (3, 0), colors.black),
        ('ALIGN', (0, 0), (3, 0), 'CENTER'),
        ('FONTNAME', (0, 0), (3, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (3, 0), 12),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    
    elements.append(household_table)
    elements.append(Spacer(1, 24))
    
    # Income verification
    elements.append(Paragraph("INCOME VERIFICATION", styles['Heading2']))
    
    income = student_info.get('income', random.randint(5000, 35000))
    tax_return_filed = random.choice([True, True, True, False])  # 75% filed taxes
    
    elements.append(Paragraph(f"Did you file a tax return for {datetime.datetime.now().year - 1}? {'Yes' if tax_return_filed else 'No'}", styles['Normal']))
    
    if tax_return_filed:
        elements.append(Paragraph(f"Adjusted Gross Income: ${income}", styles['Normal']))
        elements.append(Paragraph(f"Taxes Paid: ${int(income * 0.12)}", styles['Normal']))
        elements.append(Paragraph(f"Income Earned from Work: ${int(income * 0.9)}", styles['Normal']))
    else:
        elements.append(Paragraph(f"Reason for not filing: {random.choice(['Not required to file', 'Extension requested', 'Other circumstances'])}", styles['Normal']))
        elements.append(Paragraph(f"Income Earned from Work: ${income}", styles['Normal']))
    
    elements.append(Spacer(1, 24))
    
    # Certification
    elements.append(Paragraph("CERTIFICATION", styles['Heading2']))
    elements.append(Paragraph("By signing this worksheet, I certify that all the information reported is complete and correct to the best of my knowledge.", styles['Normal']))
    elements.append(Spacer(1, 24))
    
    elements.append(Paragraph(f"Student Signature: ___________________ Date: {fake.date_this_year().strftime('%m/%d/%Y')}", styles['Normal']))
    if "Parent" in [row[2] for row in household_data]:
        elements.append(Paragraph(f"Parent Signature: ___________________ Date: {fake.date_this_year().strftime('%m/%d/%Y')}", styles['Normal']))
    
    # Build PDF
    doc.build(elements)
    
    return buffer.getvalue()

In [10]:
def generate_appeal_letter(student_info):
    buffer = BytesIO()
    doc = SimpleDocTemplate(buffer, pagesize=letter)
    styles = getSampleStyleSheet()
    
    # Content elements
    elements = []
    
    # Date and address
    today = datetime.datetime.now().strftime("%B %d, %Y")
    elements.append(Paragraph(today, styles['Normal']))
    elements.append(Spacer(1, 12))
    
    elements.append(Paragraph("Office of Financial Aid", styles['Normal']))
    elements.append(Paragraph("University Name", styles['Normal']))
    elements.append(Paragraph("123 University Avenue", styles['Normal']))
    elements.append(Paragraph("City, State 12345", styles['Normal']))
    elements.append(Spacer(1, 24))
    
    # Subject
    elements.append(Paragraph("Subject: Financial Aid Appeal", styles['Normal']))
    elements.append(Spacer(1, 12))
    
    # Greeting
    elements.append(Paragraph("Dear Financial Aid Committee,", styles['Normal']))
    elements.append(Spacer(1, 12))
    
    # Body of letter
    appeal_reason = random.choice([
        "change in financial circumstances due to job loss",
        "unexpected medical expenses",
        "family emergency situation",
        "death in the family",
        "natural disaster damage to home"
    ])
    
    paragraph1 = f"I am writing to appeal my financial aid award for the {datetime.datetime.now().year}/{datetime.datetime.now().year + 1} academic year. My name is {names[student_info['student_id']]} and my student ID is {student_info['student_id']}. I was notified of my financial aid award, but I am requesting a reconsideration due to a {appeal_reason} that has significantly impacted my family's ability to contribute to my education."
    
    paragraph2 = fake.paragraph(nb_sentences=4)
    
    paragraph3 = "Due to these circumstances, I am respectfully requesting a reconsideration of my financial aid package. Any additional assistance would greatly help me continue my education without excessive financial hardship."
    
    paragraph4 = "I have attached the following supporting documentation:"
    
    documents = []
    if appeal_reason == "change in financial circumstances due to job loss":
        documents.extend(["Termination letter", "Unemployment benefit statement"])
    elif appeal_reason == "unexpected medical expenses":
        documents.extend(["Medical bills", "Insurance statements"])
    elif appeal_reason == "family emergency situation" or appeal_reason == "death in the family":
        documents.extend(["Relevant certificates", "Statement from family member"])
    elif appeal_reason == "natural disaster damage to home":
        documents.extend(["Insurance claim", "Repair estimates"])
    
    paragraph5 = "Thank you for your time and consideration of my appeal. I am committed to my education and look forward to continuing my studies. Please contact me at any time if you need additional information."
    
    elements.append(Paragraph(paragraph1, styles['Normal']))
    elements.append(Spacer(1, 12))
    elements.append(Paragraph(paragraph2, styles['Normal']))
    elements.append(Spacer(1, 12))
    elements.append(Paragraph(paragraph3, styles['Normal']))
    elements.append(Spacer(1, 12))
    elements.append(Paragraph(paragraph4, styles['Normal']))
    
    for docu in documents:
        elements.append(Paragraph(f"• {docu}", styles['Normal']))
    
    elements.append(Spacer(1, 12))
    elements.append(Paragraph(paragraph5, styles['Normal']))
    elements.append(Spacer(1, 24))
    
    # Closing
    elements.append(Paragraph("Sincerely,", styles['Normal']))
    elements.append(Spacer(1, 24))
    elements.append(Paragraph(names[student_info['student_id']], styles['Normal']))
    elements.append(Paragraph(f"Email: {emails[student_info['student_id']]}", styles['Normal']))
    elements.append(Paragraph(f"Phone: {fake.phone_number()}", styles['Normal']))
    
    # Build PDF
    doc.build(elements)
    
    return buffer.getvalue()

In [11]:
def generate_payment_plan(student_info):
    buffer = BytesIO()
    doc = SimpleDocTemplate(buffer, pagesize=letter)
    styles = getSampleStyleSheet()
    
    # Content elements
    elements = []
    
    # Title
    elements.append(Paragraph("STUDENT PAYMENT PLAN AGREEMENT", styles['Title']))
    
    # Student information
    elements.append(Paragraph("STUDENT INFORMATION", styles['Heading2']))
    elements.append(Paragraph(f"Name: {names[student_info['student_id']]}", styles['Normal']))
    elements.append(Paragraph(f"Student ID: {student_info['student_id']}", styles['Normal']))
    elements.append(Paragraph(f"Email: {emails[student_info['student_id']]}", styles['Normal']))
    elements.append(Paragraph(f"Phone: {fake.phone_number()}", styles['Normal']))
    elements.append(Spacer(1, 12))
    
    # Term information
    current_year = datetime.datetime.now().year
    term = random.choice(["Fall", "Spring", "Summer"])
    elements.append(Paragraph(f"Term: {term} {current_year}", styles['Heading2']))
    
    # Financial details
    total_due = random.randint(5000, 25000)
    down_payment = int(total_due * random.uniform(0.1, 0.25))
    remaining_balance = total_due - down_payment
    
    num_payments = random.choice([3, 4, 5])
    payment_amount = remaining_balance / num_payments
    
    financial_data = [
        ["Description", "Amount"],
        ["Total Amount Due", f"${total_due}"],
        ["Down Payment", f"${down_payment}"],
        ["Remaining Balance", f"${remaining_balance}"],
        ["Number of Payments", str(num_payments)],
        ["Payment Amount", f"${int(payment_amount)}"]
    ]
    
    financial_table = Table(financial_data, colWidths=[300, 100])
    financial_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (1, 0), colors.lightgrey),
        ('TEXTCOLOR', (0, 0), (1, 0), colors.black),
        ('ALIGN', (0, 0), (1, 0), 'CENTER'),
        ('FONTNAME', (0, 0), (1, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (1, 0), 12),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    
    elements.append(financial_table)
    elements.append(Spacer(1, 24))
    
    # Payment schedule
    elements.append(Paragraph("PAYMENT SCHEDULE", styles['Heading2']))
    
    # Calculate payment dates
    start_date = datetime.datetime.now()
    if term == "Fall":
        start_date = datetime.datetime(current_year, 9, 1)
    elif term == "Spring":
        start_date = datetime.datetime(current_year, 1, 15)
    else:  # Summer
        start_date = datetime.datetime(current_year, 6, 1)
    
    payment_dates = []
    for i in range(num_payments):
        payment_date = start_date + datetime.timedelta(days=30 * (i + 1))
        payment_dates.append(payment_date.strftime("%m/%d/%Y"))
    
    schedule_data = [["Payment Number", "Due Date", "Amount"]]
    for i, date in enumerate(payment_dates):
        schedule_data.append([f"Payment {i+1}", date, f"${int(payment_amount)}"])
    
    schedule_table = Table(schedule_data, colWidths=[150, 150, 100])
    schedule_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (2, 0), colors.lightgrey),
        ('TEXTCOLOR', (0, 0), (2, 0), colors.black),
        ('ALIGN', (0, 0), (2, 0), 'CENTER'),
        ('FONTNAME', (0, 0), (2, 0), 'Helvetica-Bold'),
        ('BOTTOMPADDING', (0, 0), (2, 0), 12),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))
    
    elements.append(schedule_table)
    elements.append(Spacer(1, 24))
    
    # Terms and conditions
    elements.append(Paragraph("TERMS AND CONDITIONS", styles['Heading2']))
    
    terms = [
        "Payments are due on or before the scheduled due dates.",
        "A late fee of $25 will be assessed for payments made after the due date.",
        "Failure to make payments may result in a hold on registration, transcripts, and diplomas.",
        "This payment plan does not exempt the student from any financial obligations as stated in the university's policies.",
        "The student is responsible for notifying the Bursar's Office of any changes that may affect this agreement."
    ]
    
    for term in terms:
        elements.append(Paragraph(f"• {term}", styles['Normal']))
    
    elements.append(Spacer(1, 24))
    
    # Signatures
    elements.append(Paragraph("SIGNATURES", styles['Heading2']))
    elements.append(Paragraph("By signing below, I agree to the terms and conditions of this payment plan:", styles['Normal']))
    elements.append(Spacer(1, 24))
    
    elements.append(Paragraph(f"Student Signature: ___________________ Date: {fake.date_this_year().strftime('%m/%d/%Y')}", styles['Normal']))
    elements.append(Spacer(1, 12))
    elements.append(Paragraph("University Official: ___________________ Date: ___________________", styles['Normal']))
    
    # Build PDF
    doc.build(elements)
    
    return buffer.getvalue()

In [12]:
def generate_external_scholarship(student_info):
    buffer = BytesIO()
    doc = SimpleDocTemplate(buffer, pagesize=letter)
    styles = getSampleStyleSheet()
    
    # Content elements
    elements = []
    
    # Organization header
    organization_names = [
        "Community Foundation",
        "Local Business Association",
        "Regional Education Fund",
        "Professional Association",
        "Alumni Scholarship Trust",
        "Industry Excellence Fund"
    ]
    organization = random.choice(organization_names)
    
    # Date and address
    today = datetime.datetime.now().strftime("%B %d, %Y")
    elements.append(Paragraph(f"{organization}", styles['Title']))
    elements.append(Paragraph(f"{fake.street_address()}", styles['Normal']))
    elements.append(Paragraph(f"{fake.city()}, {fake.state()} {fake.zipcode()}", styles['Normal']))
    elements.append(Paragraph(f"Phone: {fake.phone_number()}", styles['Normal']))
    elements.append(Paragraph(f"Email: scholarships@{organization.lower().replace(' ', '')}.org", styles['Normal']))
    elements.append(Spacer(1, 24))
    
    elements.append(Paragraph(today, styles['Normal']))
    elements.append(Spacer(1, 12))
    
    elements.append(Paragraph(f"{names[student_info['student_id']]}", styles['Normal']))
    elements.append(Paragraph(f"{fake.street_address()}", styles['Normal']))
    elements.append(Paragraph(f"{fake.city()}, {fake.state()} {fake.zipcode()}", styles['Normal']))
    elements.append(Spacer(1, 24))
    
    # Subject
    award_names = [
        "Merit Scholarship",
        "Community Leadership Award",
        "Academic Achievement Scholarship",
        "Future Leaders Fund",
        "Professional Development Grant"
    ]
    award_name = random.choice(award_names)
    elements.append(Paragraph(f"Subject: {award_name} Award Notification", styles['Heading2']))
    elements.append(Spacer(1, 12))
    
    # Greeting
    elements.append(Paragraph(f"Dear {names[student_info['student_id']].split()[0]},", styles['Normal']))
    elements.append(Spacer(1, 12))
    
    # Body of letter
    award_amount = random.choice([1000, 1500, 2000, 2500, 3000, 5000])

    contents = [
        "Your outstanding academic performance, leadership qualities, and commitment to personal growth have distinguished you among a highly competitive group of applicants. The scholarship committee was particularly impressed by your dedication to your field of study and your clear vision for the future. This award recognizes not only your past accomplishments but also our confidence in your continued success. Congratulations once again on this well-deserved recognition. We wish you the very best in the coming year.",
        "This scholarship recognizes your hard work, determination, and the potential you have demonstrated throughout your academic journey. We understand that pursuing higher education involves many challenges, and we hope this financial assistance will help ease some of that burden. Your application stood out for its heartfelt personal statement and your clear commitment to making a positive impact in your community. If you have any questions regarding the award or next steps, do not hesitate to reach out. We wish you continued success in your studies.",
        "Your dedication to academic excellence and community service reflects the values that this scholarship was designed to honor. We believe you are a rising leader who will make a meaningful contribution in your chosen field. We encourage you to carry this momentum forward and continue being a role model to your peers. Congratulations on your success!",
        "To receive the scholarship, please ensure that you remain enrolled full-time and maintain a minimum GPA as previously discussed. Should you require any assistance or have questions about the disbursement process, please contact the Financial Aid Office. Congratulations on your achievement—we’re excited to support you in your continued success.",
        "We hope this helps relieve some financial pressure and allows you to focus more fully on your studies and passions. Don’t forget to take time to celebrate this accomplishment—you’ve earned it! We look forward to seeing the great things you’ll continue to achieve. Best of luck in the new semester!"
    ]
    
    paragraph1 = f"We are pleased to inform you that you have been selected to receive the {award_name} in the amount of ${award_amount} for the {datetime.datetime.now().year}/{datetime.datetime.now().year + 1} academic year. After careful review of all applications, our selection committee has determined that you exemplify the qualities and achievements this award was established to recognize."
    paragraph2 = random.choice(contents)
    elements.append(Paragraph(paragraph1, styles['Normal']))
    elements.append(Spacer(1, 12))
    elements.append(Paragraph(paragraph2, styles['Normal']))
    elements.append(Spacer(1, 24))
    
    # Closing
    elements.append(Paragraph("Sincerely,", styles['Normal']))
    elements.append(Spacer(1, 24))
    elements.append(Paragraph("Scholarship Program Coordinator", styles['Normal']))
    elements.append(Paragraph(fake.name(), styles['Normal']))

    # Build PDF
    doc.build(elements)
    
    return buffer.getvalue()

In [13]:
PATH = '/Users/ngochoang/Library/CloudStorage/GoogleDrive-nhungoc1508@gmail.com/My Drive/Graduate/Semester 2 (Spring 2025)/Big Data Management/Project/P1 - Landing zone/S25-BDM-Project/data-simulation/data/finance'

In [14]:
# award_content = generate_external_scholarship(student_info)
# file_path = f'{PATH}/award.pdf'
# with open(file_path, 'wb') as f:
#     f.write(award_content)

In [15]:
functions = [generate_tax_return, generate_fafsa,
             generate_scholarship_application,
             generate_verification_document,
             generate_appeal_letter, generate_payment_plan,
             generate_external_scholarship]

In [19]:
NUM_RECORDS = 1000

In [20]:
for _ in range(NUM_RECORDS):
    student_id = random.choice(student_ids)
    student_info = {"student_id": student_id}
    func = random.choice(functions)
    content = func(student_info)
    file_id = str(uuid.uuid4())
    file_path = f'{PATH}/{file_id}.pdf'
    with open(file_path, 'wb') as f:
        f.write(content)